# Overview on worldwide carbon emissions and electricity sources

#### Analysis on worldwide CO2 emissions, the impact different electricity sources have on the carbon footprint of a country and how they can help smooth sudden energy prices movements.

### 1.0 Purpose of this analysis
This analysis can be divided into two parts.
In the first part, the goal is to assess the status of ___worldwide CO2 emissions___ as of 2023, as well as the impact of ___various electricity production sources___ on a country's carbon footprint and how these trends evolved over time. Particular focus has been dedicated to CO2 emissions from combustion processes (in mln tons) rather than countries' total emissions, as the former are more relevant to electricity production processes.    
In the last sections, the information from the first part is utilized to assess how the energy mix of a country can help smooth the impact of sudden movements in energy's market prices.

### 2.0 Overview of the data  
The dataset presented in this analysis has been sourced from the Energy Institute website, and includes information on carbon emissions and electricity production from various sources, mainly fossil fuels, nuclear energy, renewables and other sources.    
The data is a panel data, providing insight on countries from all over the globe and across various years. It has been however chosen to focus on the year 2023, the latest year available, to provide a quick overview of the most recent information on the matter.   
An overview of the most relevant attributes is provided below, after selecting the features to be included in the analysis.


In [1]:
import sys
print(sys.executable)

c:\Users\enric\Documents\carbon-footprint\.venv\Scripts\python.exe


In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import plotly.graph_objects as go
import plotly.express as px
import requests
import os
import warnings
import seaborn as sns
import polars
from scipy.stats import pearsonr
from scipy.stats import ks_2samp
from sklearn.preprocessing import PowerTransformer
from utils import *

warnings.filterwarnings('ignore')

In [3]:
# Download and save data
panelUrl = "https://www.energyinst.org/__data/assets/excel_doc/0007/1055752/merged_panel.xlsx"
panelFile = "Data/Panel Data.xlsx"

glossaryUrl = "https://www.energyinst.org/__data/assets/excel_doc/0020/1540505/Consolidated-Dataset-Narrow-Format-Glossary.xlsx"
glossaryFile = "Data/Glossary.xlsx"

panelDataDowloader = DownloadSave(panelUrl, panelFile)
panelData = panelDataDowloader.downloadSave()

glossaryDownloader = DownloadSave(glossaryUrl, glossaryFile)
glossaryData = glossaryDownloader.downloadSave()

panelData.head()
print(glossaryData.to_string(line_width=None))

                      Code                                           Variable                                       Units
0     biodiesel_cons_kboed                             biodiesel consumption   thousand barrels of oil equivalent per day
1        biodiesel_cons_pj                             biodiesel consumption                                   Petajoules
2     biodiesel_prod_kboed                               biodiesel production  thousand barrels of oil equivalent per day
3        biodiesel_prod_pj                               biodiesel production                                  Petajoules
4         biofuels_cons_ej                               biofuels consumption                                   Exajoules
5        biofuels_cons_kbd                               biofuels consumption                    thousand barrels per day
6      biofuels_cons_kboed                               biofuels consumption  thousand barrels of oil equivalent per day
7         biofuels_cons_

### 3.0 Data Cleaning and Feature Engineering

#### 3.1 Data Cleaning

Most of the data cleaning process involves cleaning the 'Country' column from values that represented aggregate information for other countries and, thus, provided no additional information for our purposes. Therefore, these rows have been removed from the dataset.  
Finally, a low number of missing values has been observed regarding carbon emissions and primary energy consumption columns. All missing values, including those in electricity production features, are preserved here, as possibly indicators of a country's specific energy policy or transition towards greener sources. In other words, missing information on coal electricity production for a specific country does not necessarily imply a defect in the data, but can be interpreted as the country transitioning towards other energy sources.

In [4]:
# keep only relevant columns
selectedColumns = ['Country', 'pop', 'Year', 'Region', 'OPEC', 'EU', 'OECD', 'CIS',
                   'co2_combust_mtco2', 'co2_combust_pc', 'co2_combust_per_ej', 'co2_mtco2', 'elect_twh']
electByFuel = panelData.filter(like = "electbyfuel")
primaryEnergyCons = panelData.filter(like = "primary_")
panelDataFiltered = pd.concat([panelData[selectedColumns], electByFuel, primaryEnergyCons], axis = 1)

glossaryData = glossaryData[glossaryData['Code'].isin(panelDataFiltered.columns.tolist())]
newRows = [
    {'Code':'Country', 'Variable':'Name of each country, for 107 total nations', 'Units':'-'},
    {'Code':'Region', 'Variable':'Region to which each country belongs to', 'Units':'-'},
    {'Code':'OPEC', 'Variable':'1 if is an OPEC country, 0 otherwise', 'Units':'-'},
    {'Code':'EU', 'Variable':'1 if is an EU country, 0 otherwise', 'Units':'-'},
    {'Code':'OECD', 'Variable':'1 if is an OECD country, 0 otherwise', 'Units':'-'},
    {'Code':'CIS', 'Variable':'1 if is a CIS* country, 0 otherwise', 'Units':'-'}
]
newRows = pd.DataFrame(newRows)
glossaryData = pd.concat([glossaryData, newRows], ignore_index = True)
# print the list of features used in the analysis
glossaryData.style.set_table_attributes('style="width:100%; display:block; overflow:auto;"').set_table_styles([
    {'selector': 'thead th', 'props': [('text-align', 'center')]},  # Center align headers
    {'selector': 'tbody td', 'props': [('text-align', 'center')]},   # Center align data
])

print(glossaryData.to_string(line_width=None))
print("*Note - CIS: Commonwealth of Independent States")

panelDataFiltered = panelDataFiltered[~panelDataFiltered['Country'].str.contains('^Total', na = False)]
panelDataFiltered = panelDataFiltered[~panelDataFiltered['Country'].str.contains('^Other', na = False)]
panelDataFiltered = panelDataFiltered[panelDataFiltered['Country'] != 'Rest of World']

# Create Column w/ CO2 emissions per capita
panelDataFiltered['co2_mtco2_pc'] = panelDataFiltered['co2_mtco2']/panelDataFiltered['pop']

# Extract data as of 2023
energyData = panelDataFiltered.copy()
energyData = energyData[energyData.co2_mtco2.notnull()]
energyData = energyData[energyData['Year'] == 2023]

# Create column for international organization
condList = [energyData['OPEC'] == 1, energyData['EU'] == 1, 
            energyData['CIS'] == 1, energyData['OECD'] == 1]
choiceList = ['OPEC', 'EU', 'CIS', 'OECD']
energyData['InternationalOrganization'] = np.select(condList, choiceList, default = 'None')

                     Code                                           Variable                      Units
0       co2_combust_mtco2                      CO2 emissions from combustion     Million tonnes of CO2 
1          co2_combust_pc                      CO2 emissions from combustion  CO2 combustion per capita
2      co2_combust_per_ej                                   Carbon intensity          CO2 per exajoules
3               co2_mtco2                               Total CO2 emissions      Million tonnes of CO2 
4               elect_twh                                        Electricity              Terawatt-hour
5        electbyfuel_coal                   Electricity generation from coal              Terawatt-hour
6         electbyfuel_gas                    Electricity generation from gas              Terawatt-hour
7       electbyfuel_hydro                  Electricity generation from hydro              Terawatt-hour
8     electbyfuel_nuclear                Electricity generation 

In [5]:
#print all numeric columns
print(energyData.select_dtypes(include='float64').columns.tolist())

['pop', 'OPEC', 'EU', 'OECD', 'CIS', 'co2_combust_mtco2', 'co2_combust_pc', 'co2_combust_per_ej', 'co2_mtco2', 'elect_twh', 'electbyfuel_coal', 'electbyfuel_gas', 'electbyfuel_hydro', 'electbyfuel_nuclear', 'electbyfuel_oil', 'electbyfuel_other', 'electbyfuel_ren_power', 'electbyfuel_total', 'primary_eintensity', 'primary_ej', 'primary_ej_pc', 'co2_mtco2_pc']


In [6]:
# SKEWNESS

# Skewness of data
numCols = energyData.select_dtypes(include = ['float64']).columns.tolist()
print(energyData[numCols].skew().sort_values(ascending = False))

transformer = TreatSkewedVariables(energyData, 0.75)
energyDataTransformed = transformer.YeoJohnson()

# While it is important to normalize the distributions in order to conduct meaningful statistical tests,
# It is also important to preserve outliers, as they indicate which are the most relevant global polluters
# One action that can be done is to look at the percentage contribution of each country to the global CO2 emissions

co2_combust_mtco2        6.322900
co2_mtco2                6.298515
elect_twh                6.214241
primary_ej               5.788529
pop                      5.134280
electbyfuel_coal         4.996785
electbyfuel_oil          4.698540
electbyfuel_gas          4.642247
electbyfuel_ren_power    4.267929
electbyfuel_total        4.193242
electbyfuel_hydro        3.943784
co2_combust_pc           3.487231
co2_mtco2_pc             3.319303
electbyfuel_other        3.275147
CIS                      3.025012
primary_eintensity       2.773503
OPEC                     2.725420
electbyfuel_nuclear      2.701736
primary_ej_pc            2.650319
EU                       0.892103
OECD                     0.235148
co2_combust_per_ej      -0.319341
dtype: float64


In [7]:
# print(energyData.select_dtypes(include = 'float64').columns.tolist)
print(glossaryData[glossaryData['Code'].isin(energyData.filter(like = 'electby').columns)].to_string(line_width=None))
print(energyData.filter(like = 'electby').columns.tolist)
globalCO2Emissions = energyData['co2_combust_mtco2'].sum()
energyData['co2_perc_contr_total'] = energyData['co2_combust_mtco2']/globalCO2Emissions

energyData['elect_fossil'] = energyData[['electbyfuel_coal','electbyfuel_gas','electbyfuel_oil']].sum(axis=1, numeric_only=True)
energyData['elect_renew'] = energyData['electbyfuel_ren_power']
energyData['elect_nuclear'] = energyData['electbyfuel_nuclear']
energyData['elect_sustain'] = energyData[['electbyfuel_ren_power','electbyfuel_nuclear']].sum(axis=1, numeric_only=True)
energyData['elect_other'] = energyData['electbyfuel_other']

energyData['elect_fossil_Unskew'] = energyData[['electbyfuel_coal_Unskew', 'electbyfuel_gas_Unskew', 'electbyfuel_oil_Unskew']].sum(axis = 1, numeric_only = True)
energyData['elect_nuclear_Unskew'] = energyData['electbyfuel_nuclear_Unskew']
energyData['elect_renew_Unskew'] = energyData['electbyfuel_ren_power_Unskew']
energyData['elect_other_Unskew'] = energyData['electbyfuel_other_Unskew']
energyData['elect_sustain_Unskew'] = energyData[['elect_nuclear_Unskew', 'elect_renew_Unskew']].sum(axis = 1, numeric_only = True)

electSources = ['elect_fossil', 'elect_renew', 'elect_nuclear', 'elect_sustain', 'elect_other']

for col in electSources:
    newCol = col + '_perc'
    energyData[newCol] = energyData[col].div(energyData['electbyfuel_total'])
    energyData[newCol + '_Unskew'] = energyData[col + '_Unskew'].div(energyData['electbyfuel_total_Unskew'])

# What are the primary sources of energy of the world?
energyData['co2_perc_Unskew'] = energyData['co2_combust_mtco2_Unskew']/energyData['co2_combust_mtco2_Unskew'].sum()


                     Code                                Variable          Units
5        electbyfuel_coal        Electricity generation from coal  Terawatt-hour
6         electbyfuel_gas         Electricity generation from gas  Terawatt-hour
7       electbyfuel_hydro       Electricity generation from hydro  Terawatt-hour
8     electbyfuel_nuclear     Electricity generation from nuclear  Terawatt-hour
9         electbyfuel_oil         Electricity generation from oil  Terawatt-hour
10      electbyfuel_other       Electricity generation from other  Terawatt-hour
11  electbyfuel_ren_power  Electricity generation from renewables  Terawatt-hour
12      electbyfuel_total            Total electricity generation  Terawatt-hour
<bound method IndexOpsMixin.tolist of Index(['electbyfuel_coal', 'electbyfuel_gas', 'electbyfuel_hydro',
       'electbyfuel_nuclear', 'electbyfuel_oil', 'electbyfuel_other',
       'electbyfuel_ren_power', 'electbyfuel_total', 'electbyfuel_coal_Unskew',
       'electbyf

In [8]:
# OUTLIERS
# Detect outliers with z-scores approach
OutliersDetect = DetectOutliers(df=energyData)
a = energyData['co2_mtco2_Unskew'].quantile(.95)
OutliersDetect.z_score(column='co2_mtco2_Unskew', threshold=a)
# 10 outliers detected - which countries?
print(energyData[energyData['co2_mtco2_Unskew_outlier'] == 1][['Country','co2_mtco2','co2_mtco2_Unskew','co2_mtco2_Unskew_z_score']].sort_values('co2_mtco2').to_string(line_width=None))
OutliersViz = DataVisualizer(df = energyData.sort_values('co2_mtco2'))
OutliersViz.BarChart(xcol='Country', ycol='co2_mtco2')
OutliersViz.show()

                 Country     co2_mtco2  co2_mtco2_Unskew  co2_mtco2_Unskew_z_score
3362     North Macedonia      8.223451         -2.282774                 -2.282774
5042            Slovenia     10.902435         -2.050917                 -2.050917
2701           Lithuania     12.337375         -1.949044                 -1.949044
1571             Estonia     15.132630         -1.780958                 -1.780958
1135             Croatia     17.538715         -1.659874                 -1.659874
6675   Trinidad & Tobago     20.426463         -1.535308                 -1.535308
4814  Russian Federation   2176.075218          1.727223                  1.727223
2048               India   3121.547344          1.931543                  1.931543
6895                  US   5130.147811          2.202767                  2.202767
983                China  12603.514979          2.665112                  2.665112


In [9]:
# Look at emissions per capita
a = energyData['co2_mtco2_pc_Unskew'].quantile(.95)
OutliersDetect.z_score(column='co2_mtco2_pc_Unskew', threshold=a)
print(energyData['co2_mtco2_pc_Unskew_z_score'].describe())
print(energyData['co2_mtco2_pc_Unskew_outlier'].sum())
print('Outliers detected:')
print(energyData[energyData['co2_mtco2_pc_Unskew_outlier'] == 1][['Country','co2_mtco2','co2_mtco2_pc']].sort_values('co2_mtco2_pc').to_string(line_width=None))

count    7.000000e+01
mean    -9.516197e-18
std      1.007220e+00
min     -2.688981e+00
25%     -5.419065e-01
50%     -8.961740e-02
75%      5.100759e-01
max      2.544449e+00
Name: co2_mtco2_pc_Unskew_z_score, dtype: float64
7
Outliers detected:
                   Country   co2_mtco2  co2_mtco2_pc
450             Bangladesh  124.601872      0.720432
4313              Pakistan  221.975428      0.923030
4446           Philippines  163.997776      1.397660
2564                Kuwait  125.755959     29.176985
6836          Turkmenistan  222.203193     34.100642
7023  United Arab Emirates  340.792127     35.809262
4623                 Qatar  164.241321     60.463063


### 4.0 Exploratory Data Analysis

#### 4.1 Carbon emissions by region  
In the plots below, the distribution of carbon emissions from combustion processes is presented, both in total million tons as well as per capita. Please note that China, India and United States have been excluded from the first two plots. This is because their values lie significantly above other countries that compressed remaining distributions, thus making the resulting plots less interpretable. Their distribution is presented in separate barplots.  
The ___boxplots___ below evince the presence of outliers, despite the removal of three of the largest global emittors.  
* For _total CO2 emissions_, outliers are mostly present for Asia Pacific, Europe and CIS.
* For _per capita emissions_, outliers are mostly present for Asia Pacific, Europe and CIS.
* Final observation that can be drawn from the boxplots below is the wider distribution of Middle East per capita emissions compared to its total emissions. This can be due to most of fossil fuels reserves being present in the Middle East, Russia and the US. This would imply higher emissions for Middle Eastern countries than other similar countries, without a significantly higher population.

In [20]:
DataViz = DataVisualizer(energyData.sort_values('electbyfuel_total'), theme='plotly_white')
DataViz.LineChart(xcol='electbyfuel_total', ycol='co2_combust_mtco2')
DataViz.show()
# DataViz.ScatterPlot(xcol='electbyfuel_total', ycol='co2_combust_mtco2')
# DataViz.show()
# DataViz.BoxPlot(ycol = 'co2_combust_mtco2')
# DataViz = DataVisualizer(energyData.sort_values('co2_mtco2'), theme='plotly_white')
# DataViz.BarChart(xcol='Country', ycol='co2_mtco2', title='Total CO2 emissions (Million tons)')
# DataViz.show()
# DataViz = DataVisualizer(energyData.sort_values('co2_combust_mtco2'), theme='plotly_white')
# DataViz.BarChart(xcol='Country', ycol='co2_combust_mtco2', title='CO2 (Million tons) emissions from combustion processes')
# DataViz.show()

In [33]:
SubPlotsViz = SubPlots(df = energyData.sort_values('co2_mtco2'))
SubPlotsViz.BarChartSubPlots(rows = 1, cols=2, xcol='Country', ycol=('co2_mtco2','co2_mtco2_pc'))